In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [101 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launc

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-15 01:32:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-15 01:32:32 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-08-31 00:00:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

In [5]:
# Converting timestamp type to date type for required column
from pyspark.sql.functions import *
df = df.withColumn("review_date", to_date(col("review_date"),"yyyy-MM-dd"))
df.show(10)
df.printSchema()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [6]:
# Printing number of records in dataset
df = df.dropna()
df = df.dropDuplicates()
print(df.count())

3093660


In [7]:
# Creating dataframe for review_id_table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.printSchema()

review_id_table.show(10)

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10049SWV3MPMV|   19690777|B009IKCJ8E|     709267596| 2014-02-13|
|R100QDK1V60OML|   16523479|B000MXWSYQ|     327623395| 2013-09-10|
|R100SHHDOK086N|   11310466|B00737FWLK|     779320587| 2013-05-30|
|R100ZY5I4ABLVE|   14503757|B000068O4N|     734576678| 2013-11-21|
|R10183KOXFU6GE|   41702406|B0045VVX1A|     929407244| 2014-09-29|
|R101BBO6SJBA4T|   13574421|B00I053I5G|     244828807| 2015-04-07|
|R101YA1KGZOPR5|    6659043|B001MQ8EB8|     944454860| 2014-09-16|
|R1025X0DH2KX2O|   32315100|B0058TKX2I|     666494366| 2011-12-09|
|R10295X58E7A0F|   33589220|B00HTPHK5W|   

In [8]:
# Creating products table
product_gb = df.groupby("product_id", "product_title").count()

In [9]:
products = product_gb.select("product_id", "product_title")

In [10]:
# Creating customers table
customer_gb = df.groupby("customer_id").count()

In [11]:
customers = customer_gb.withColumnRenamed("count", "customer_count")
customers = customers.withColumn("customer_count", customers.customer_count.cast('int'))
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37890218|             2|
|   11374463|             1|
|   33031518|             1|
|   16431454|             2|
|   17920715|             2|
|   13846972|             3|
|   31561079|             1|
|   43332274|             1|
|   23947515|             4|
|   14124780|             2|
|   26687886|            26|
|   47483454|             1|
|   19505799|             1|
|   12607147|             3|
|   51437479|             2|
|   51904488|             1|
|   19248825|             2|
|   42038329|             5|
|   16762670|             1|
|   27582160|            23|
+-----------+--------------+
only showing top 20 rows



In [12]:
# Creating vine table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url=""
config = {"user":"", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [14]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table2', mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table='products2', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers2', mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table='vine_table2', mode=mode, properties=config)